In [ ]:
#clean data in excel file
import pandas as pd
import re

def check_excel_data(df, price_columns, text_columns):
    print("=== Basic Info ===")
    print(df.info())
    print("\n")

    print("=== Missing Values per Column ===")
    print(df.isnull().sum())
    print("\n")

    for col in price_columns:
        cleaned = df[col].astype(str).str.replace(r'[^\d.]', '', regex=True)
        non_numeric_mask = ~cleaned.str.match(r'^\d*\.?\d*$')
        bad_entries = df.loc[non_numeric_mask, col]
        print(f"=== Problematic entries in '{col}' ===")
        if not bad_entries.empty:
            print(bad_entries)
        else:
            print("No invalid entries found.")
        print("\n")

    for col in text_columns:
        missing_text = df[col].isnull().sum() + (df[col].astype(str).str.strip() == '').sum()
        print(f"=== Missing or empty entries in '{col}': {missing_text}")

        short_text = df[df[col].astype(str).str.len() < 3]
        print(f"=== Rows with unusually short text in '{col}':")
        if not short_text.empty:
            print(short_text[[col]])
        else:
            print("None found.")

        def has_weird_chars(text):
            return bool(re.search(r'[^a-zA-Z0-9\s,.\-\'\"]', text))

        weird_text = df[df[col].astype(str).apply(has_weird_chars)]
        print(f"=== Rows with unusual characters in '{col}':")
        if not weird_text.empty:
            print(weird_text[[col]])
        else:
            print("None found.")
        print("\n")

    print("=== Numeric Summary ===")
    print(df.describe(include='all'))


filename = "amazon_product_reviews.xlsx"
df = pd.read_excel(filename)

price_columns = ['discounted_price', 'actual_price']
text_columns = ['product_name', 'category', 'about_product', 'user_name', 'review_title', 'review_content']
rating_columns = ['rating', 'rating_count']

check_excel_data(df, price_columns, text_columns)

for col in price_columns:
    df[col] = df[col].astype(str).str.replace(r'[^\d.]', '', regex=True)
    df[col] = pd.to_numeric(df[col], errors='coerce')

for col in rating_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

print("\n=== Data Info After Cleaning ===")
print(df.info())


=== Basic Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   product_id           1465 non-null   object 
 1   product_name         1465 non-null   object 
 2   category             1465 non-null   object 
 3   discounted_price     1465 non-null   object 
 4   actual_price         1465 non-null   object 
 5   discount_percentage  1465 non-null   float64
 6   rating               1465 non-null   object 
 7   rating_count         1463 non-null   object 
 8   about_product        1465 non-null   object 
 9   user_id              1465 non-null   object 
 10  user_name            1465 non-null   object 
 11  review_id            1465 non-null   object 
 12  review_title         1465 non-null   object 
 13  review_content       1465 non-null   object 
 14  img_link             1465 non-null   object 
 15  product_link       

In [ ]:
#Create Database from the excel file 
import pandas as pd
import sqlite3        

df = pd.read_excel("amazon_product_reviews.xlsx")
conn = sqlite3.connect('exceldb.db')  
df.to_sql('Customer', conn, if_exists='replace', index=False)
conn.close()
print("SQLite database created successfully from Excel!")


SQLite database created successfully from Excel!
